# AVIRIS: sample zarr access

Sample zarr files were produced from downloaded tar archives using [av3-to-zarr](av3-to-zarr.ipynb) notebook.

Needs a fairly new version of `odc-geo`

```
pip install "odc-geo>=0.4.7"
```

In [ ]:
import numpy as np
import odc.geo.xr  # needed for .odc. accessor
import xarray as xr

## Xarray.open_zarr

When loading data with xarray it's important to use this configuration:

```python
xarray.open_zarr(..., decode_coords="all")
```

Keep in mind that load is "lazy", no actual pixels are loaded yet, but spatial metadata is available so one can focus on a specific region and only load that part.

In [ ]:
urls = [
    "s3://adias-prod-dc-data-projects/odc-hs/av3/AV320230915t213013_L2A_OE_main_98b13fff.zarr",
    "s3://adias-prod-dc-data-projects/odc-hs/av3/AV320230915t214314_L2A_OE_main_98b13fff.zarr",
    "s3://adias-prod-dc-data-projects/odc-hs/av3/AV320230915t214955_L2A_OE_main_98b13fff.zarr",
]
xxs = [xr.open_zarr(url, decode_coords="all") for url in urls]

xx = xxs[0]  # Pick first one for review
display(xx.odc.geobox, xx.odc.spatial_dims, xx.odc.transform, xx)

## Review Infrared Band nearest to 1000nm

### Load into RAM

In [ ]:
%%time
ir = xx.rfl.sel(wavelength=1000, method="nearest").rename("ir").compute()

### Plot on a Map

In [ ]:
ir.odc.explore(name="Infrared", robust=True, cmap="OrRd")

## Review Visible Spectrum

Produce RGBA image from bands nearest to 650, 550 and 450nm and display it on a map.

### Load into RAM

In [ ]:
%%time
rgb = xx.rfl.sel(wavelength=[650, 550, 450], method="nearest").compute()

### Convert to RGBA

- Compute alpha mask
- Clip top and bottom 2% of pixel values
- Remap remaining 96% of valid pixels into `[0, 255]` range for visual display
- Form RGBA image using `uint8` pixels

In [ ]:
# Transparency image in uint8
alpha = xr.where(rgb[..., 0].isnull(), np.uint8(0), np.uint8(255))

# RGB normalized to 0->1
rmin, rmax = np.nanpercentile(rgb.data, [2, 98])
rgb = ((rgb - rmin) / (rmax - rmin)).clip(0, 1)

# RGBA (uint8: [0->255])
rgba = xr.concat(
    [(256 * rgb).clip(0, 255).astype("uint8"), alpha],
    dim="wavelength",
).rename("rgba")

### Plot on a Map

- Display RGBA image
- Add footprints to the map

In [ ]:
from folium import GeoJson

_map = rgba.odc.explore(resampling="bilinear", name=f"{xx.id} RGBA")

# add footprints to the map
for x in xxs:
    gbox = x.odc.geobox
    GeoJson(
        gbox.extent.geojson(),
        tooltip=f"{x.id}",
        control=False,
        name=f"{x.id}",
    ).add_to(_map, index=2)

display(_map)

## Review Single Pixel Location

Xarray currently lacks proper geospatial index. So let's define `latlon_query` that translates a single point specified in lat lon into a pixel location that xarray can understand.

In [ ]:
from odc.geo.geom import point

def latlon_query(lat, lon, src, method="nearest", **kw):
    pt = point(lon, lat, 4326)
    gbox = src.odc.geobox

    if gbox.axis_aligned:
        pt = pt.to_crs(gbox.crs)
    else:
        # need to convert to image coordinates
        pt = gbox.project(pt)

    x, y = pt.points[0]
    return {"x": x, "y": y, "method": method, **kw}

### Define Point Query

- Define query location
- Add it to a map from before
- Display the map again

In [ ]:
from folium import Marker

lat, lon = 38.229, -122
Marker(
    [lat, lon],
    name="Query",
    tooltip=f"lat: {lat:.3f}, lon: {lon:.3f}",
    control=False,
).add_to(_map, index=2)
display(_map)

### Load and Plot

In [ ]:
pix = xx.rfl.sel(**latlon_query(lat, lon, xx)).compute()
_ = pix.plot()

# Appendix

## List available zarrs on S3

In [ ]:
!aws s3 ls s3://adias-prod-dc-data-projects/odc-hs/av3/ | grep ".zarr"

---------------------------------